In [ ]:
from google.colab import files
import pandas as pd

# Upload the dataset
uploaded = files.upload()

# Assuming the uploaded files are "Test_Medicine Dataset - Sheet1.csv" and "sample_prescription.csv"
medicine_dataset_filename = "Test_Medicine Dataset - Sheet1 (1).csv"
prescription_filename = "sample_prescription.csv"

# Load the datasets into pandas DataFrames
medicine_dataset = pd.read_csv(medicine_dataset_filename)
prescription_df = pd.read_csv(prescription_filename)

# Display the first few rows and columns to check the names
print("Medicine Dataset loaded successfully.")
print(medicine_dataset.head())
print(medicine_dataset.columns)

print("Prescription Dataset loaded successfully.")
print(prescription_df.head())
print(prescription_df.columns)

# Function to analyze a single medication
def analyze_medication(prescription, medication, dataset):
    liver_disease = prescription["Liver_disease"].lower()
    renal_egfr = prescription["Renal_eGFR"]
    dose_type = prescription["Dose_Type"].lower()
    dosage = float(prescription["Dosage"])
    pregnancy = prescription["Pregnancy"].lower()
    pregnancy_stage = prescription["Pregnancy_stage"].lower()
    glucose_test_type = prescription["Glucose_test_type"].lower()
    glucose_level = prescription["Glucose_Level"]

    # Filter the dataset for the given medication
    medication_data = dataset[dataset["Medication"].str.contains(medication, case=False, na=False)]

    if medication_data.empty:
        return f"\nMedication {medication} not found in the dataset."

    # Pregnancy check
    if pregnancy == 'yes':
        generic_name = medication_data.iloc[0]["Generic"]
        if generic_name.lower() == "metformin hydrochloride":
            if pregnancy_stage == "before birth":
                if medication_data.iloc[0]["Pregnancy_Before Birth"].lower() in ["not use", "not recommended"]:
                    return f"\nFound Issue in Recommended Medication {medication}! \nBecause the Mentioned Medication is not being used for Pregnancy Stage: Before Birth."
            elif pregnancy_stage == "after birth":
                if medication_data.iloc[0]["Pregnancy_After Birth"].lower() in ["not use", "not recommended"]:
                    return f"\nFound Issue in Recommended Medication {medication}! \nBecause the Mentioned Medication is not Recommended for Pregnancy Stage: After Birth."

    # Glucose test type and level check
    if glucose_test_type == 'random blood sugar test' and float(glucose_level) < 200:
        return f"\nFound Issue in Recommended Dosing for {medication}! \nBecause the Glucose Level is not Appropriate."
    elif glucose_test_type == 'a1c test' and float(glucose_level) < 6.5:
        return f"\nFound Issue in Recommended Dosing for {medication}! \nBecause the Glucose Level is not Appropriate."
    elif glucose_test_type == 'fasting blood sugar test' and float(glucose_level) < 126:
        return f"\nFound Issue in Recommended Dosing for {medication}! \nBecause the Glucose Level is not Appropriate."

    # Liver disease medications not recommended
    liver_disease_meds_not_recommended = ["D-MET 500", "D-MET 850", "D-FO 850", "Diabest 500", "Diabest 850",
                                          "Diabex 500", "Diabex 850", "Dia M 500", "Dia M 850", "Dia M SR 500",
                                          "Dialiptin-M 500", "Dialiptin M 850", "DMF 500", "DMF 850",
                                          "Empaglif-M 10/100 XR Tablet", "Empaglif-M 12.5/1000 Tablet",
                                          "Empaglif-M 12.5/500 Tablet", "Empaglif-M 25/1000 XR Tablet",
                                          "Empaglif-M 5/1000 Tablet", "Empagliflozin + Metformin Hydrochloride (multiple brands)",
                                          "Empanor M", "Empazin M 100", "Empazin M 12.5/500", "Empazin M 500",
                                          "EmpaMet 12.5/1000 Tablet", "EmpaMet 12.5/500 Tablet", "EmpaMet 5/1000",
                                          "EmpaMet 5/850 Tablet", "EmpaMet XR 10/1000 Tablet", "EmpaMet XR 25/1000 Tablet",
                                          "EmpaMet XR 5/1000 Tablet", "Emaglin - M 5/500", "EMPATAB M 5/500 TABLET",
                                          "Etform 500", "Etform 850", "Formet 500", "Formet 850", "Formet 1 gm XR",
                                          "Formet XR 500", "Formet XR 750", "Formet-L", "Formet-V", "G-Metformin",
                                          "G-Metformin 850 mg", "G Phase 850", "G-Phase 500", "Glunil 500", "Glunil 850",
                                          "Glucil", "Glucomin 500", "Glucomin 850", "Glumin 2.5/250", "Glumin PLUS 5/500",
                                          "Glunor 500", "Glunor 850", "Glunor OD 1 G Extended Release", "Glunor OD 500 Extended Release",
                                          "Glunor XR 500", "Gluvan Plus 500", "Gluvan Plus 850", "GOODMET 500", "GOODMET 850",
                                          "Himet 500", "Himet 850", "Informet 500", "Informet 850", "Informet LA 500",
                                          "Informet LA 750", "Informet LA 1000", "Info-500 500", "Info-850 850", "Kemin 500",
                                          "Kemin 850", "Kemin SR 500", "Leamet 500", "Leamet 850", "Lemet 500", "Lemet 850",
                                          "Lim 500", "Lim 850", "Linagliptin + Metformin Hydrochloride (multiple brands)",
                                          "Linaptin M 500", "Linaptin M 850", "Linarol-M 500", "Linarol-M 850", "Linarol-M 1000",
                                          "Linatab M 2.5/1000", "Linatab M 2.5/1000 ER", "Linatab M 2.5/850", "Linatab M 2.5/500",
                                          "Linax Plus 2.5/1000", "Linax Plus 2.5/500", "Linax Plus 5/1000", "Linax Plus (other dosages)",
                                          "Lijenta-M 1000", "Lijenta-M 500", "Lijenta-M 850", "Lijenta"]

    # Check the Liver disease criteria
    if liver_disease == 'yes' and medication in liver_disease_meds_not_recommended:
        return f"\nFound Issue in Recommended Medication {medication}. \nMentioned Medication is not Recommended for Patient having Liver Disease!!"
    elif liver_disease == 'yes' and medication not in liver_disease_meds_not_recommended:
        return f"\nFound Issue in Recommended Medication {medication}. \nMentioned Medication are not being used for Patient having Liver Disease!!"

    # Check the Renal eGFR, dose type, and dosage within the minimum and maximum range
    for index, row in medication_data.iterrows():
        min_dose = float(row["Minimum Dose"].replace('mg', '').strip())
        max_dose = float(row["Maximum Dose"].replace('mg', '').strip())
        if row["Dose Type"].lower() == dose_type and row["Renal eGFR Test"].lower() == renal_egfr.lower() and min_dose <= dosage <= max_dose:
            return f"\nPrescription for {medication} is Ok!!"

    return f"\nFound Issue in Recommended Dosing for {medication}! \nPrescription Change recommended."

# Function to analyze prescription from CSV
def analyze_prescription_from_csv(prescription_df, dataset):
    results = []
    for index, prescription in prescription_df.iterrows():
        medications = prescription["Medication"].split(', ')
        for medication in medications:
            result = analyze_medication(prescription, medication, dataset)
            results.append(result)
    return "\n".join(results)

# Run the analysis
result = analyze_prescription_from_csv(prescription_df, medicine_dataset)
print(result)
